# Mandelbrot Set

Loading in from packages

In [ ]:
# Import standard packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time

# Trange for tracking ciomputations
from tqdm import trange

# Toolkits for drawing figures
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

Settings for the notebook

In [ ]:
create_baseline = False

Class for the mandelbrot set, leading for the entire notebook

In [ ]:
import numpy as np
import pickle

class MandelBrot():

    """
    Class which helps creating a standard grid on which a mandelbrot can be
    sampled. It consists of methods for creating a full mandelbrot, which has 
    the sole purpose of drawing images, and multiple sampling methods for the
    grid.
    """

    def __init__(self, create_full=False, load=False, max_iterations=100, X=(-2, 1), Y=(-1.5, 1.5), MaxSampleSize=1024):

        """
        Initilization of the class.

        args:
            create_full:    (Bool)  Set to true if image has to be drawn
            load:           (Bool)  Set to true if mandelbrotdata has to be read from memory
            max_iterations: (Int)   Maximum iterations for the full mandelbrot
            X:              (Tuple) Min and max of the grid
            Y:              (Tuple) Min and max of the grid
            MaxSampleSize:  (Int)   Decisive for the amount of grid blocks on the X and Y axis

        returns:
            MandelbrotSet   (Class)
        """

        self.sharpness = (X[1] - X[0])/ MaxSampleSize
        self.X_grid = np.arange(X[0], X[1], self.sharpness)
        self.Y_grid = np.arange(Y[0], Y[1], self.sharpness)
        self.grid_size = (len(self.X_grid), len(self.Y_grid))
        self.max_iterations = max_iterations
        self.area_size = (X[1] - X[0]) * (Y[1] - Y[0])

        self.MaxSampleSize = len(self.X_grid)
        
        if create_full == True:
            self._create_MandelBrot()
        elif load == True:
            self._load_Mandelbrot()

    def _iterate(self, complex_number, max_iterations):
        """
        Method to iterate using complex numbers;
        
        args:
            complex_number: (Complex) Complex number
            max_iterations: (Int)     Max iterations to check for convergence
        
        returns:
            interations:    (Int)     Amount of iterations done for divergence
        """
        zn = 0
        for i in range(max_iterations):
            if abs(zn) > 2:
                break
            zn = zn**2 + complex_number
        return i + 1

    def _create_MandelBrot(self, save=True):
        """ 
        Method for creating full mandelbrotset with as purpose drawing a fig.

        args:
            save:   (Bool)  Boolean to save the created class as a pickle object
        """
        self.Z = np.zeros(self.grid_size)
        for i in range(np.shape(self.Z)[0]):
            for j in range(np.shape(self.Z)[1]):
                # print(x, i, y, j)
                complex_number = complex(self.X_grid[i], self.Y_grid[j])
                iters = self._iterate(complex_number, self.max_iterations)
                self.Z[j][i] = 1 - iters/self.max_iterations
        
        if save:
            dbfile = open('mandelbrotdata_class', 'wb')
            pickle.dump(self.Z, dbfile)                     
            dbfile.close()
    
    def _load_Mandelbrot(self):
        """ 
        Method for loading data from a pickle object. 
        """
        infile = open("mandelbrotdata",'rb')
        self.Z = pickle.load(infile)
        self.X_grid, self.Y_grid = np.shape(self.Z)
        infile.close()

    def RANDOM_SAMPLE(self, N):
        """ 
        Method to RANDOM sample the created grid. 

        args:
            N:      (Int)   Amount to be sampled.
        returns:
            samples (Tuple) A tuple of N * X samples and N * Y samples. 
                            Being a total of N coordinates.
        """
        # No more samples than the grid divided spaces can be taken
        if N > self.MaxSampleSize:
            print(f'Sample size to big! Should be lower than {self.MaxSampleSize}')
            return
        else:
            return (
                np.random.choice(self.X_grid, size=N, replace=True),
                np.random.choice(self.Y_grid, size=N, replace=True)
            )

    
    def LHC_SAMPLE(self, N):
        """ 
        Method to Latin Hyper Cube (LHC) sample the created grid. 

        args:
            N:      (Int)   Amount to be sampled.
        returns:
            samples (Tuple) A tuple of N * X samples and N * Y samples. 
                            Being a total of N coordinates.
        """
        # No more samples than the grid divided spaces can be taken
        if N > self.MaxSampleSize:
            print(f'Sample size to big! Should be lower than {self.MaxSampleSize}')
            return
        else:
            return (
                np.random.choice(self.X_grid, size=N, replace=False),
                np.random.choice(self.Y_grid, size=N, replace=False)
            )

    def ORTHOGONAL_SAMPLE(self, N, grid_divide=4):
        """ 
        Method to ORTHOGONAL sample the created grid. 

        args:
            N:              (Int)   Amount to be sampled.
            grid_divide:    (Int)   The amount of spaces the X and Y 
                                    grid has to be divided in. Total 
                                    amount of squares is grid_divide**2
        returns:
            samples:        (Tuple) A tuple of N * X samples and N * Y samples. 
                                    Being a total of N coordinates.
        """
        # No more samples than the grid divided spaces can be taken
        if N > self.MaxSampleSize:
            print(f'Sample size to big! Should be lower than {self.MaxSampleSize}')
            return
        else:
            # Create divisions for x and y axis.
            stepsize = int(len(self.X_grid)/grid_divide)
            self.stepsize = stepsize
            x_choices, y_choices = [], []

            # Create arrays that keep track of the columns and rows that are already selected
            x_masked, y_masked = np.ones(len(self.X_grid), dtype='bool'), np.ones(len(self.Y_grid), dtype='bool')

            counter = 0
            # While there are still possible samples left
            while ((np.any(x_masked) == True) & 
                   (np.any(y_masked) == True)):
                # if counter > 10:
                #     print(x_masked, y_masked)
                #     print(x_choices, 'X') 
                #     print(y_choices, 'Y')
                #     return

                counter+=1  
                for i_y in range(0, len(x_masked), stepsize):
                    for i_x in range(0, len(x_masked), stepsize):

                        try:
                            x_choice = np.random.choice(np.arange(i_x, i_x + stepsize, 1)[x_masked[i_x: i_x + stepsize]])
                            y_choice = np.random.choice(np.arange(i_y, i_y + stepsize, 1)[y_masked[i_y: i_y + stepsize]])

                            x_masked[x_choice] = False
                            x_choices.append(x_choice)

                            y_masked[y_choice] = False
                            y_choices.append(y_choice)
                        except:
                            pass

                        # Break out inner for loop when choices surpass desired sample size
                        if len(x_choices) >= N:
                            break
                    # Break from middle for loop when choices surpass desired sample size
                    if len(x_choices) >= N:
                            break
                
                # Break out outer for loop when choices surpass desired sample size
                if len(x_choices) >= N:
                            break

            return self.X_grid[x_choices], self.Y_grid[y_choices]



# Monte Carlo Method

In [ ]:
def monte_carlo(samples, max_iterations, mandelbrot):
    """
    Performs the monte carlo simulation.

    args:
        samples:        (Int)   Total N samples given to check for convergence
        max_iterations: (Int)   Maximum amount of iterations to check for convergence
        mandelbrot:     (Class) Mandelbrot set as created in the MandelBrot Class
    
    returns:
        area_estimation (Float) Estimation of the mandelbrot area
    """
    
    # counters
    Inner = 0
    
    for i in range(len(samples[0])):
        iters = mandelbrot._iterate(complex(samples[0][i], samples[1][i]), max_iterations)
        if iters >= max_iterations:
            Inner += 1

    return mandelbrot.area_size * (Inner/ len(samples[0]))

In [ ]:
def calc_error(estimation, baseline=1.510995):
    """ 
    Method to calculate error based on the baseline

    args:
        estimation: (Float) Estimation of the mandelbrotset
        baseline:   (Float) Baseline as calculated in the previous methods
    """
    return abs(estimation - baseline)
    
def estimate_area(mandelbrotset, samplesize, iterations, minimal_runs=30, method='random'):
    """
    Method to estimate area for a samplesize, iteration and runs.

    args:
        mandelbrotset   (Class)         Mandelbrotset class as created above
        samplesize      (Int)           Amount of samples
        iterations:     (Int)           Iterations done to check for convergence
        minimal_runs    (Int)           N of runs to calculate mean and variance of the samplesize 
                                        and iterations given
        method:         (Class Method)  Class method which is responsible for the sampling method

    returns:
        MeanEstimation: (Float)         Mean of all estimations done for the given samplesize 
                                        and iterations
        MeanError:      (Float)         Mean of all errors done 
        variance:       (Float)         Variance of the estimations
        stdev:          (Float)         Standard deviationf of the estimations
    """
    AllEstimations, AllErrors = [], []
    counter, stdev, d = 1, 100, 1.96
    while ((counter <= minimal_runs) |
          ((stdev/np.sqrt(counter)) > d)):

        if method.lower() == 'random':
            samples = mandelbrotset.RANDOM_SAMPLE(N=samplesize)
        elif method.lower() == 'lhc':
            samples = mandelbrotset.LHC_SAMPLE(N=samplesize)
        elif method.lower() == 'orthogonal':
            samples = mandelbrotset.ORTHOGONAL_SAMPLE(N=samplesize)
        else:
            print('No known method selected.\nPerforms random sampling.')
            samples = mandelbrotset.RANDOM_SAMPLE(N=samplesize)

        area_estimation = monte_carlo(samples, iterations, mandelbrotset)
        error = calc_error(area_estimation)

        AllEstimations.append(area_estimation), AllErrors.append(error)

        if counter > 1:
            variance = np.var(AllEstimations, ddof=1)
            stdev = np.sqrt(variance)

        if counter > 100:
            print('Calculation not finished within confidence interval of 95%\n')
            break

        counter += 1

    return [np.mean(AllEstimations), np.mean(AllErrors), variance, stdev]

def estimate_area_boxplot(mandelbrotset, samplesize, iterations, runs, method='random'):
    """
    Method to estimate area for a samplesize, iteration and runs for the purpose 
    of creating the boxplots.

    args:
        mandelbrotset   (Class)         Mandelbrotset class as created above
        samplesize      (Int)           Amount of samples
        iterations:     (Int)           Iterations done to check for convergence
        runs            (Int)           Runs
        method:         (Class Method)  Class method which is responsible for the sampling method

    returns:
        MeanEstimation: (Float)         Mean of all estimations done for the given samplesize 
                                        and iterations
        MeanError:      (Float)         Mean of all errors done 
    """
    AllEstimations, AllErrors, AllMethods = [], [], []

    for _ in trange(runs):

        if method.lower() == 'random':
            samples = mandelbrotset.RANDOM_SAMPLE(N=samplesize)
        elif method.lower() == 'lhc':
            samples = mandelbrotset.LHC_SAMPLE(N=samplesize)
        elif method.lower() == 'orthogonal':
            if samplesize == 3025:
                samples = mandelbrotset.ORTHOGONAL_SAMPLE(N=samplesize, grid_divide=5)
            else:
                samples = mandelbrotset.ORTHOGONAL_SAMPLE(N=samplesize)
        elif method.lower() == 'orthogonal+':
            samples = mandelbrotset.ORTHOGONAL_SAMPLE(N=samplesize, grid_divide=np.sqrt(samplesize))
        else:
            print('No known method selected.\nPerforms random sampling.')
            samples = mandelbrotset.RANDOM_SAMPLE(N=samplesize)

        area_estimation = monte_carlo(samples, iterations, mandelbrotset)
        error = calc_error(area_estimation)

        AllEstimations.append(area_estimation), AllErrors.append(error), AllMethods.append(method)

    return (AllEstimations, AllErrors, AllMethods)

# Creation of baseline

In [ ]:
if create_baseline == True:
    # Create Baseline predictions
    BASE_MANDELBROT = MandelBrot(X=(-2, 1), Y=(-1.5, 1.5), MaxSampleSize=1e5)

    # Create empty lists to save estimations
    random_ests, lhc_ests, orthogonal_ests = [], [], []

    # Create 30 samples
    print('Baseline loop starting..')
    for i in trange(30):

        # Remove output to check progress
        clear_output(wait=True)

        # random_ests.append(monte_carlo(BASE_MANDELBROT.RANDOM_SAMPLE(int(1e5)), 1000, BASE_MANDELBROT))
        # lhc_ests.append(monte_carlo(BASE_MANDELBROT.LHC_SAMPLE(int(1e5)), 1000, BASE_MANDELBROT))
        orthogonal_ests.append(monte_carlo(BASE_MANDELBROT.ORTHOGONAL_SAMPLE(int(1e5)), 1000, BASE_MANDELBROT))

        print('Current progress: ', np.round(((i+1)/30)*100, 2), "%")

    # Save the baselines
    baselines = {
        'Random': np.mean(random_ests),
        'LHC': np.mean(lhc_ests),
        'Orthogonal': np.mean(orthogonal_ests)
    }

    # remove garbage list to save memory
    del random_ests
    del lhc_ests
    del orthogonal_ests


In [ ]:
# # Save baselines
# dbfile = open('baselines', 'wb')
# pickle.dump(baselines, dbfile)                     
# dbfile.close()

In [ ]:
# # Load baselines
# infile = open('baselines', 'rb')
# baseline = pickle.load(infile)['Orthogonal']
# infile.close()
# del infile

# All experiments

In [ ]:
# Create the grid from wich the values will be sampled.
MyMandelbrot = MandelBrot(
    create_full=False,
    load=False,  
    MaxSampleSize=3e4
)
print('Gridsize_X: ' f'{np.min(MyMandelbrot.X_grid)}, {np.max(MyMandelbrot.X_grid)}')
print('Gridsize_Y: ' f'{np.min(MyMandelbrot.Y_grid)}, {np.max(MyMandelbrot.Y_grid)}')
print('Sharpness: ', MyMandelbrot.sharpness)
print('Gridsize: ', MyMandelbrot.grid_size)

# Create empty values for saving output
output = []
methods = ['Random', 'LHC', 'Orthogonal']
n_samples = np.arange(1, 1001, 1)
n_iterations = np.arange(1, 301, 1)

# Save the fixed values for testing
GIVEN_ITER = 100
GIVEN_SAMPLESIZE = 1000

# Create output for multiple samplesizes
print('Looping over samplesizes...')
for i in trange(len(n_samples)):
    for method in methods:
        row = estimate_area(MyMandelbrot, 
                            samplesize=n_samples[i], 
                            iterations=GIVEN_ITER,
                            method=method)
        row.extend([n_samples[i], GIVEN_ITER, method])
        output.append(row)

# Save to dataframe
df_samplesizes = pd.DataFrame(output, columns=['AreaEstimation', 'Error', 'Variance', 'StDev',
                                               'SampleSize', 'N_iterations', 'Method'])

# Create output for multiple iterations
output = []
print("Looping over iterations possibilities...")
for i in trange(len(n_iterations)):
    for method in methods:
        row = estimate_area(MyMandelbrot,
                            samplesize=GIVEN_SAMPLESIZE, 
                            iterations=n_iterations[i], 
                            method=method)
        row.extend([GIVEN_SAMPLESIZE, n_iterations[i], method])
        output.append(row)

# Save to dataframe
df_iterations = pd.DataFrame(output, columns=['AreaEstimation', 'Error', 'Variance', 'StDev',
                                              'SampleSize', 'N_iterations', 'Method'])


del output
    

In [ ]:
# Create grid for valeus
MyMandelbrot = MandelBrot(
    create_full=False,
    load=False,  
    MaxSampleSize=3025
)
print('Gridsize_X: ' f'{np.min(MyMandelbrot.X_grid)}, {np.max(MyMandelbrot.X_grid)}')
print('Gridsize_Y: ' f'{np.min(MyMandelbrot.Y_grid)}, {np.max(MyMandelbrot.Y_grid)}')
print('Sharpness: ', MyMandelbrot.sharpness)
print('Gridsize: ', MyMandelbrot.grid_size)

# Give the samplesize and the iterations
SAMPLESIZE = 3025
N_ITERATIONS = 100

AllEstimations, AllErrors, AllMethods = [], [], []
methods = ['random', 'LHC', 'orthogonal', 'orthogonal+']
# methods = ['orthogonal', 'orthogonal+']

# Loop over methods and perform 10.000 tests
for method in methods:
    print(f"Looping for {method} sampling...")
    output = estimate_area_boxplot(MyMandelbrot, 
                                samplesize=SAMPLESIZE, 
                                iterations=N_ITERATIONS, # dan moet je hiervan n_iterations[i] maken 
                                method=method, 
                                runs=1000)
    AllEstimations.extend(output[0]) 
    AllErrors.extend(output[1])
    AllMethods.extend(output[2])
        
# Save data
df_boxplot = pd.DataFrame()
df_boxplot['AreaEstimation'] = AllEstimations
df_boxplot['Error'] = AllErrors
df_boxplot['Method'] = AllMethods
df_boxplot


In [ ]:
# Some code was ran on other computer, so load in
infile = open('new_grid_data_1637590099.620807', 'rb')
_ = pickle.load(infile)
infile.close()
np.shape(_[0])

In [ ]:
# Create df for boxplots
new_df = pd.DataFrame()
for method in np.unique(df_boxplot['Method']):
    new_df[method] = df_boxplot.loc[df_boxplot['Method'] == method]['AreaEstimation'].to_list()

new_df['intelligent random'] = _[1]
new_df['intelligent LHC'] = _[3]

# Figures and statistics

In [ ]:
import seaborn as sns

sns.set(style='darkgrid')

fig, axis = plt.subplots(1, 1, figsize=(18, 6))
sns.boxplot(data=new_df, order=['random', 'intelligent random', 'LHC', 'intelligent LHC', 'orthogonal', 'orthogonal+'])
axis.tick_params('both', labelsize=14)
axis.set_xlabel('Method', fontsize=16)
axis.set_ylabel('Area Estimation', fontsize=16)
fig.suptitle('Area estimation for the different sampling methods', fontsize=20)


In [ ]:
# perform levene test for all possible combinations

from scipy.stats import levene

methods = new_df.columns
for method in methods:
       for method2 in methods:
              stat, p = levene(new_df[method], new_df[method2]))
       methods = np.delete(methods, np.argwhere(methods == method))

In [ ]:
new_df.to_csv('final_boxplot.csv')

In [ ]:
from scipy.optimize import curve_fit

# Functions for fitting line
def func(x, a, b, c):
    return a * -b * np.sqrt(x) /  x - c

def fit_curve(df, name):
    popt, pcov = curve_fit(func, df[name], df['Error'])
    return popt

# Draw figure for report
def draw_simple_figure(df, what='samplesizes'):
    if what == 'samplesizes':
        name = 'SampleSize'
    else:
        name = 'N_iterations'
    
    methods = ['Random', 'LHC', 'Orthogonal']
    colors = ['red', 'blue', 'green']


    fig, ax = plt.subplots(figsize=(18, 10))
    
    # set limits of figure
    ax.set_xlim(0,1000)
    ax.set_ylim(-.5, 2)
    ax.hlines(0, 0, 1000, color='black', linewidth=0.8)

    # Zoom in on interesting part
    axins = zoomed_inset_axes(ax, 2, loc=1)
    axins.hlines(0, 0, 200, color='black', linewidth=0.5)

    # plot all points
    for i in range(len(methods)):
        _ = df.loc[df['Method'] == methods[i]]
        if name == 'SampleSize':
            popt = fit_curve(_, name)
            ax.plot(_[name], func(_[name], *popt), c=colors[i])
        ax.scatter(_[name], _['Error'], s=1, c=colors[i], label=methods[i])
        ax.fill_between(_[name], _['Error'] + _['StDev'], y2=_['Error'] - _['StDev'], alpha=0.15, color=colors[i])

        axins.scatter(_[name], _['Error'], s=10, c=colors[i])
        axins.fill_between(_[name], _['Error'] + _['StDev'], y2=_['Error'] - _['StDev'], alpha=0.15, color=colors[i])
        axins.plot(_[name], func(_[name], *popt), c=colors[i])

    # set limits
    axins.set_xlim(750, 950)
    axins.set_ylim(-0.1, 0.4)

    # Change tick params
    axins.tick_params(axis='x', 
                    which='both', 
                    bottom=False, 
                    top=True, 
                    labelbottom=False, 
                    labeltop=True)
    axins.tick_params(axis='y', 
                    which='both', 
                    left=False,
                    right=True, 
                    top=False, 
                    labelright=True, 
                    labelleft=False)

    # Draw the zooming lines
    mark_inset(ax, axins, loc1=3, loc2=4, fc='none', ec='0.2')

    # Change tick parkams
    ax.tick_params('both', labelsize=16)
    axins.tick_params('both', labelsize=14)

    # give the figure his grid and legend
    ax.legend(loc=2, fontsize=16, markerscale=10)
    ax.grid(alpha=0.5)

    # Labels
    ax.set_xlabel('Sample sizes', fontsize=18)
    ax.set_ylabel('Mean error', fontsize=18)

    # set title
    plt.suptitle('The mean error and standard deviation for different sample sizes', fontsize=24)
    plt.show()

draw_simple_figure(df_samplesizes, what='samplesizes')

In [ ]:
from scipy.optimize import curve_fit

# Function for drwaing figure with zoomed in version
def draw_simple_figure(df, what='samplesizes'):
    if what == 'samplesizes':
        name = 'SampleSize'
    else:
        name = 'N_iterations'
    
    methods = ['Random', 'LHC', 'Orthogonal']
    colors = ['red', 'blue', 'green']


    fig, ax = plt.subplots(figsize=(18, 10))
    
    # set limits of figure
    ax.set_xlim(0,300)
    ax.set_ylim(-.5, 2)
    ax.hlines(0, 0, 300, color='black', linewidth=0.8)

    # Zoom in on interesting part
    axins = zoomed_inset_axes(ax, 2, loc=1)
    axins.hlines(0, 200, 275, color='black', linewidth=0.8)
    
    # Plot all points
    for i in range(len(methods)):
        _ = df.loc[df['Method'] == methods[i]]
        if name == 'samplesizes':
            popt = fit_curve(_, name)
            ax.plot(_[name], func(_[name], *popt), c=colors[i])
            axins.plot(_[name], func(_[name], *popt), c=colors[i])
        ax.scatter(_[name], _['Error'], s=1, c=colors[i], label=methods[i])
        ax.fill_between(_[name], _['Error'] + _['StDev'], y2=_['Error'] - _['StDev'], alpha=0.15, color=colors[i])

        axins.scatter(_[name], _['Error'], s=10, c=colors[i])
        axins.fill_between(_[name], _['Error'] + _['StDev'], y2=_['Error'] - _['StDev'], alpha=0.15, color=colors[i])

    # Set limits  
    axins.set_xlim(200, 275)
    axins.set_ylim(-0.1, 0.4)

    # Change tick params
    axins.tick_params(axis='x', 
                    which='both', 
                    bottom=False, 
                    top=True, 
                    labelbottom=False, 
                    labeltop=True)
    axins.tick_params(axis='y', 
                    which='both', 
                    left=False,
                    right=True, 
                    top=False, 
                    labelright=True, 
                    labelleft=False)

    # Draw lines for zooming
    mark_inset(ax, axins, loc1=3, loc2=4, fc='none', ec='0.2')

    # Change tick params
    ax.tick_params('both', labelsize=16)
    axins.tick_params('both', labelsize=14)

    # give the figure his grid and legend
    ax.legend(loc=2, fontsize=16, markerscale=10)
    ax.grid(alpha=0.5)

    # Labels
    ax.set_xlabel('Number of iterations', fontsize=18)
    ax.set_ylabel('Mean error', fontsize=18)

    # Set title
    plt.suptitle('The mean error and standard deviation for different iterations', fontsize=24)
    plt.show()

draw_simple_figure(df_iterations, what='N_iterations')

In [ ]:
import matplotlib.pyplot as plt

# Creating orthogonal sampling
x = np.arange(0, 36, 1)
y = np.arange(0, 36, 1)
stepsize = int(np.sqrt(len(x)))
print(stepsize**2)

# empty lists for saving
x_choices = []
y_choices = []

# Keep track of choices
x_masked = np.ones(len(x), dtype='bool')
y_masked = np.ones(len(y), dtype='bool')

# Loop over rows and columns
for i_y in range(0, len(x), stepsize):
    for i_x in range(0, len(x), stepsize):

        # Choice and remove
        x_choice = np.random.choice(np.arange(i_x, i_x + stepsize, 1)[x_masked[i_x: i_x + stepsize]])
        y_choice = np.random.choice(np.arange(i_y, i_y + stepsize, 1)[y_masked[i_y: i_y + stepsize]])

        # Keep track of x
        x_masked[x_choice] = False
        x_choices.append(x_choice)

        # Keep track of y
        y_masked[y_choice] = False
        y_choices.append(y_choice)

print("x_choices: ", x[x_choices])
print("y_choices: ", y[y_choices])

# Draw subplots for report
fig, axes = plt.subplots(1, 4, figsize=(18,5))

a_, b_ = np.random.choice(x, size=36), np.random.choice(y, size=36)
axes[0].scatter(a_, b_)
axes[0].hlines(b_, 0, 36, alpha=0.3, color='grey')
axes[0].vlines(a_, 0, 36, alpha=0.3, color='grey')
axes[0].set_title('Random Sampling', fontsize=14)

a_, b_ = np.random.choice(x, size=36, replace=False), np.random.choice(y, size=36, replace=False)
axes[1].scatter(a_, b_)
axes[1].hlines(b_, 0, 36, alpha=0.3, color='grey')
axes[1].vlines(a_, 0, 36, alpha=0.3, color='grey')
axes[1].set_title('LHC Sampling', fontsize=14)

a_ = np.array(x[x_choices]) + 0.5
b_  = np.array(y[y_choices]) + 0.5
axes[2].scatter(a_, b_)
axes[2].hlines(b_, 0, 36, alpha=0.3, color='grey')
axes[2].vlines(a_, 0, 36, alpha=0.3, color='grey')
axes[2].hlines([18], xmin=0, xmax=36, color='black')
axes[2].vlines([18], ymin=0, ymax=36, color='black')
axes[2].set_title('Orthogonal Sampling', fontsize=14)

a_ = np.array(x[x_choices]) + 0.5
b_  = np.array(y[y_choices]) + 0.5
axes[3].scatter(a_, b_)
axes[3].hlines(b_, 0, 36, alpha=0.3, color='grey')
axes[3].vlines(a_, 0, 36, alpha=0.3, color='grey')
axes[3].hlines([6, 12, 18, 24, 30], xmin=0, xmax=36, color='black')
axes[3].vlines([6, 12, 18, 24, 30], ymin=0, ymax=36, color='black')
axes[3].set_title('Orthogonal+ Sampling', fontsize=14)

for i in range(len(axes)):
    axes[i].tick_params('both', labelsize=14)
    axes[i].set_xlabel('X', fontsize=14)
axes[0].set_ylabel('Y', fontsize=14)

fig.suptitle("Multiple sampling methods", fontsize=20)

In [ ]:
from scipy.stats import chisquare

# Draw simple figure for report
def test_RNG(n_list, n_rand):
    a = np.arange(1,n_list+1,1)
    # print(a)

    b = np.random.choice(a, size=n_rand)

    fig, ax = plt.subplots(figsize=(16,8))
    plt.hist(b, bins=n_list, rwidth=0.8)
    plt.title(f"Histogram RNG, n=1e6", fontsize=18)
    plt.xlabel("Random Number", fontsize=18)
    plt.ylabel("Frequency", fontsize=18)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.subplots_adjust(bottom=0.15)
    plt.xlim(0,101)
    plt.savefig("rngtest.png", dpi=300)

    return chisquare(b)

n_list = 100
n_rand = int(1e6)

_ = test_RNG(n_list, n_rand)
print(_)